#### Simple Gen AI APP Using Langchain

In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [13]:
loader=WebBaseLoader("https://blog.langchain.com/langchain-langgraph-1dot0/")
loader

In [14]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://blog.langchain.com/langchain-langgraph-1dot0/', 'title': 'LangChain and LangGraph Agent Frameworks Reach v1.0 Milestones', 'language': 'en'}, page_content='\n\n\nLangChain and LangGraph Agent Frameworks Reach v1.0 Milestones\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain and LangGraph Agent Frameworks Reach v1.0 Milestones\n\n7 min read\nOct 22, 2025\n\n\n\n\n\nBy Sydney Runkle and the LangChain OSS team We\'re releasing LangChain 1.0 and LangGraph 1.0 — our first major versions of our open source frameworks! After years of feedback, we\'ve updated langchain to focus on the core agent loop, provide flexibility with a new concept of middleware, and upgrade model integrations with the

In [15]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [16]:
for docs in documents:
    print(f'{docs}==========END===========')

page_content='LangChain and LangGraph Agent Frameworks Reach v1.0 Milestones



















































Skip to content
















Case Studies




In the Loop




LangChain




Docs




Changelog





Sign in
Subscribe



















LangChain and LangGraph Agent Frameworks Reach v1.0 Milestones

7 min read
Oct 22, 2025' metadata={'source': 'https://blog.langchain.com/langchain-langgraph-1dot0/', 'title': 'LangChain and LangGraph Agent Frameworks Reach v1.0 Milestones', 'language': 'en'}==========END===========
page_content='By Sydney Runkle and the LangChain OSS team We're releasing LangChain 1.0 and LangGraph 1.0 — our first major versions of our open source frameworks! After years of feedback, we've updated langchain to focus on the core agent loop, provide flexibility with a new concept of middleware, and upgrade model integrations with the latest content types.These two frameworks serve different purposes:LangChain is the fastest way to build an A

In [20]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [22]:
vectorstoredb

In [24]:
## Query From a vector db
query="LangChain is the fastest way to build an ?"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"When to Use Each FrameworkLangChain lets you build and ship agents fast with high-level abstractions for common patterns, while LangGraph gives you fine-grained control for complex workflows that require customization.The best part? LangChain agents are built on LangGraph, so you're not locked in. Start with LangChain's high-level APIs and seamlessly drop down to LangGraph when you need more control. Since graphs are composable, you can mix both approaches—using agents created with create_agent inside custom LangGraph workflows as your needs evolve.Choose LangChain 1.0 for:Shipping quickly with standard agent patternsAgents that fit the default loop (model → tools → response)Middleware-based customizationHigher-level abstractions over low-level controlChoose LangGraph 1.0 for:Workflows with a mixture of deterministic and agentic componentsLong running business process automationSensitive workflows which necessitate more oversight / human in the loopHighly custom or complex"

In [40]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model=os.environ['model'])
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002CD52E743A0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002CD52E74460>, root_client=<openai.OpenAI object at 0x000002CD52E74340>, root_async_client=<openai.AsyncOpenAI object at 0x000002CD52E74850>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://embedding-model-ai.openai.azure.com/openai/v1/')

In [41]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002CD52E743A0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002CD52E74460>, root_client=<openai.OpenAI object at 0x000002CD52E74340>, root_async_client=<openai.AsyncOpenAI object at 0x000002CD52E74850>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://embeddin

In [42]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangChain is the fastest way to build an ?",
    "context":[Document(page_content="When to Use Each FrameworkLangChain lets you build and ship agents fast with high-level abstractions for common patterns, while LangGraph gives you fine-grained control for complex workflows that require customization.The best part? LangChain agents are built on LangGraph, so you're not locked in. Start with LangChain's high-level APIs and seamlessly drop down to LangGraph when you need more control. Since graphs are composable, you can mix both approaches—using agents created with create_agent inside custom LangGraph workflows as your needs evolve.Choose LangChain 1.0 for:Shipping quickly with standard agent patternsAgents that fit the default loop (model → tools → response)Middleware-based customizationHigher-level abstractions over low-level controlChoose LangGraph 1.0 for:Workflows with a mixture of deterministic and agentic componentsLong running business process automationSensitive workflows which necessitate more oversight / human in the loopHighly custom or complex")]
})

'Use LangChain 1.0 when you want to move fast with standard agent patterns and higher-level abstractions:\n- Shipping quickly with common agent workflows\n- Agents that fit the default loop (model → tools → response)\n- Middleware-based customization without low-level control\n\nUse LangGraph 1.0 when you need fine-grained control for complex or sensitive workflows:\n- Workflows mixing deterministic and agentic components\n- Long-running business process automation\n- Sensitive processes requiring oversight or human-in-the-loop\n- Highly custom or complex logic\n\nNotes from the context:\n- LangChain agents are built on LangGraph, so you can start with LangChain and drop into LangGraph when needed.\n- Graphs are composable, so you can mix both approaches (for example, use agents created with create_agent inside custom LangGraph workflows).'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [43]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [44]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [45]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002CD52982260>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [46]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangChain is the fastest way to build what ?"})
response['answer']

'Use LangChain when you want fast, high-level agent development; use LangGraph when you need fine-grained control for complex or sensitive workflows.\n\nQuick guidance (from the provided context)\n- Choose LangChain 1.0 for:\n  - Shipping quickly with standard agent patterns\n  - Agents that fit the default loop (model → tools → response)\n  - Middleware-based customization\n  - Higher-level abstractions over low-level control\n\n- Choose LangGraph 1.0 for:\n  - Workflows mixing deterministic and agentic components\n  - Long‑running business process automation\n  - Sensitive workflows requiring more oversight or human‑in‑the‑loop\n  - Highly custom or complex workflows\n\nAdditional notes from the context\n- LangChain agents are built on LangGraph, so you can start with LangChain and drop into LangGraph when you need more control.\n- Graphs are composable: you can mix approaches (e.g., use agents created with create_agent inside custom LangGraph workflows).'

In [47]:

response

{'input': 'LangChain is the fastest way to build what ?',
 'context': [Document(id='5a533a25-a615-488f-b1be-f5e6bf684f6f', metadata={'source': 'https://blog.langchain.com/langchain-langgraph-1dot0/', 'title': 'LangChain and LangGraph Agent Frameworks Reach v1.0 Milestones', 'language': 'en'}, page_content="When to Use Each FrameworkLangChain lets you build and ship agents fast with high-level abstractions for common patterns, while LangGraph gives you fine-grained control for complex workflows that require customization.The best part? LangChain agents are built on LangGraph, so you're not locked in. Start with LangChain's high-level APIs and seamlessly drop down to LangGraph when you need more control. Since graphs are composable, you can mix both approaches—using agents created with create_agent inside custom LangGraph workflows as your needs evolve.Choose LangChain 1.0 for:Shipping quickly with standard agent patternsAgents that fit the default loop (model → tools → response)Middlewa

In [30]:
response['context']

[Document(page_content='use usage limits to prevent future overspend.LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estimate:Lets start by setting limits on our production usage, since that is where the majority of spend comes from.Setting a good total traces limit\u200bPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your assumptions before setting a limit.For example:Current Load: Our gen AI application is called between 1.2-1.5 times per second, and each API request has a trace associate